# Maps in Scientific Python

Making maps is a fundamental part of geoscience research.
Maps differ from regular figures in the following principle ways:

- Maps require a *projection* of geographic coordinates on the 3D Earth to the 2D space of your figure.
- Maps often include extra decorations besides just our data (e.g. continents, country borders, etc.)

Mapping is a notoriously hard and complicated problem, mostly due to the complexities of projection.

In this lecture, we will learn about [Cartopy](https://scitools.org.uk/cartopy/docs/latest/), one of the most common packages for making maps within python. Another popular and powerful library is [Basemap](https://matplotlib.org/basemap/); however, Basemap is [going away](https://matplotlib.org/basemap/users/intro.html#cartopy-new-management-and-eol-announcement) and being replaced with Cartopy in the near future. For this reason, new python learners are recommended to learn Cartopy.

### Credit: Ryan Abernathey and Phil Elson

This notebook is extracted from [An Introduction to Earth and Environmental Data Science](https://earth-env-data-science.github.io/intro.html) by [Ryan Abernathey](https://rabernat.github.io/). Each notebook is open source and can be downloaded for testing it on your computer. 

Lots of the material in this lesson was adopted from [Phil Elson](https://pelson.github.io/)'s excellent [Cartopy Tutorial](https://github.com/SciTools/cartopy-tutorial). Phil is the creator of Cartopy and published his tutorial under an [open license](http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/), meaning that we can copy, adapt, and redistribute it as long as we give proper attribution. **THANKS PHIL and RYAN!** 👏👏👏

## Background: Projections

### Most of our media for visualization *are* flat

Our two most common media are flat:

 * Paper
 * Screen

![](https://raw.githubusercontent.com/SciTools/cartopy-tutorial/master/static/flat_medium.jpg)

### [Map] Projections: Taking us from spherical to flat

A map projection (or more commonly refered to as just "projection") is:

> a systematic transformation of the latitudes and longitudes of locations from the surface of a sphere or an ellipsoid into locations on a plane. [[Wikipedia: Map projection](https://en.wikipedia.org/wiki/Map_projection)].

### The major problem with map projections

![orange peel](https://raw.githubusercontent.com/SciTools/cartopy-tutorial/master/static/orange_peel.jpg)

 * The surface of a sphere is different to a 2D surface, therefore we *have* to cut the sphere *somewhere*
 * A sphere's surface cannot be represented on a plane without distortion.
 
There are many different projections. Each projection defines a **Coordinate Reference System (CRS)**. If you are not familiar with the main types (cylindrical, conical, etc.) you **should read** Phil's [original tutorial](https://github.com/SciTools/cartopy-tutorial/blob/master/tutorial/projections_crs_and_terms.ipynb) for a great overview of this topic **before** proceeding to the more practical sides of Cartopy usage.

### The Components of a CRS
The coordinate reference system is made up of several key components:

- **Coordinate system**: The X, Y grid upon which your data is overlayed and how you define where a point is located in space.
- **Horizontal and vertical units**: The units used to define the grid along the x, y (and z) axis.
- **Datum**: A modeled version of the shape of the Earth which defines the origin used to place the coordinate system in space.
- **Projection Information**: The mathematical equation used to flatten objects that are on a round surface (e.g. the Earth) so you can view them on a flat surface.

## Introducing Cartopy

https://scitools.org.uk/cartopy/docs/latest/

Cartopy makes use of the powerful [PROJ.4](https://proj4.org/), numpy and shapely libraries and includes an interface built on top of Matplotlib for the creation of publication quality maps.

Key features of cartopy are its *object-oriented* projection definitions, and its ability to transform points, lines, vectors, polygons and images between those projections.

### Cartopy Projections and other reference systems

In Cartopy, each projection is a Python **class**.
Most classes of projection can be configured in projection-specific ways, although Cartopy takes an opinionated stance on sensible defaults.

Let's create a [Plate Carree](https://desktop.arcgis.com/en/arcmap/10.3/guide-books/map-projections/plate-carr-e.htm) projection instance.

To do so, we need cartopy's crs module. This is typically imported as ``ccrs`` (Cartopy Coordinate Reference Systems).

In [ ]:
import cartopy.crs as ccrs
import cartopy

Cartopy's [projection list](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html#cartopy-projections) tells us that the Plate Carree projection is available with the ``ccrs.PlateCarree`` class:


**Note:** we need to *instantiate* (create an instance of) the class in order to do anything projection-y with it! 

Calling the class without arguments creates a map with the default CRS parameters:

In [ ]:
ccrs.PlateCarree()

### Drawing a map

Cartopy optionally depends upon matplotlib, and each projection knows how to create a matplotlib Axes (or AxesSubplot) that can represent itself. This means that you don't need to tell Cartopy to create the axis. This is done under the hood.

The Axes that the projection creates is a [cartopy.mpl.geoaxes.GeoAxes](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes). This Axes subclass overrides some of matplotlib's existing methods, and adds a number of extremely useful ones for drawing maps.

We'll go back and look at those methods shortly, but first, let's actually see the cartopy+matplotlib dance in action:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.axes(projection=ccrs.PlateCarree())

That was a little underwhelming, but we can see from the output that the Axes created is indeed one of those `GeoAxesSubplot` instances.

One of the most useful methods that this class adds on top of the standard matplotlib Axes class is the ``coastlines`` method. With no arguments, it will add the [Natural Earth Data](https://www.naturalearthdata.com/) ``1:110,000,000`` scale coastline data to the map.

In [ ]:
plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

We could just as equally create a matplotlib subplot with one of the many approaches that exist. For example, the ```plt.subplots``` function could be used. In this case the projection keyword must be given as a dictionary:

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
ax.coastlines()

Projection classes have options we can use to customize the map. The Plate Carree has one option, the central longitude and the parameters to set the Earth model to be used (ccrs.Globe - *unless you know what you are doing, keep the default globe model*)

In [ ]:
ccrs.PlateCarree?

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
ax.coastlines()

### Useful methods of a GeoAxes

The [cartopy.mpl.geoaxes.GeoAxes](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes) class adds a number of useful methods.

Let's take a look at:

 * [set_global](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.set_global) - zoom the map out as much as possible
 * [set_extent](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.set_extent) - zoom the map to the given bounding box
 

 * [gridlines](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.gridlines) - add a graticule (and optionally labels) to the axes
 * [coastlines](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.coastlines) - add [Natural Earth Data](https://www.naturalearthdata.com/) coastlines to the axes
 * [stock_img](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.stock_img) - add a low-resolution Natural Earth background image to the axes
 
 
 * [imshow](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.imshow.html#matplotlib.axes.Axes.imshow) - add an image (numpy array) to the axes
 * [add_geometries](https://scitools.org.uk/cartopy/docs/latest/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.add_geometries) - add a collection of geometries (Shapely) to the axes
 
### Some More Examples of Different Global Projections
The following loop displays a few projections, add a georeferenced low-res image of the Earth from space, add the coastlines, the graticule with the labels and shows the projection name. Note that the order decides the layering of the objects on the figure.

In [ ]:
projections = [ccrs.PlateCarree(),
               ccrs.Robinson(),
               ccrs.Mercator(),
               ccrs.Orthographic(),
               ccrs.InterruptedGoodeHomolosine()
              ]
for proj in projections:
    plt.figure()
    ax = plt.axes(projection=proj) # create the instance of the GeoAxes class
    ax.stock_img()
    ax.coastlines()
    ax.gridlines(draw_labels=True)
    ax.set_title(f'{type(proj)}')

### Regional Maps

To create a regional map, we use the `set_extent` method of GeoAxis to limit the size of the region. The coastline resolution can also be chosen (see next section for more details). If no resolution is given, a best guess based on the extent is set. The label position can also be adjusted using the methods from the [gridliner](https://scitools.org.uk/cartopy/docs/latest/matplotlib/gridliner.html) instance, such as the boolean methods for locating the labels 
- `bottom_labels`
- `top_labels`
- `right_labels`
- `left_labels`

In [ ]:
ax.set_extent?

In [ ]:
central_lon, central_lat = 18, -34.5
extent = [17, 22, -32, -35]
fig,ax = plt.subplots(1,2,figsize=(10,6),subplot_kw={'projection':ccrs.Orthographic(central_lon, central_lat)})
ax[0].set_extent(extent)
gl0 = ax[0].gridlines(draw_labels=True)
ax[0].coastlines(resolution='50m')
ax[1].set_extent(extent)
gl1 = ax[1].gridlines(draw_labels=True)
ax[1].coastlines(resolution='10m')
gl0.right_labels = False
gl0.top_labels = False
gl1.right_labels = False
gl1.top_labels = False

## Adding Features to the Map

To give our map more styles and details, we add `cartopy.feature` objects [(see the manual for all the features)](https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html) 
Many useful features are built in, obtained from the [Natural Earth Data](https://www.naturalearthdata.com/) data set. These "default features" are at coarse resolution, but they are scaled based on the extent of your mapped region. The dataset scale keyword is one of ‘10m’, ‘50m’, or ‘110m’, which correspond to 1:10,000,000, 1:50,000,000, and 1:110,000,000 respectively.

Name | Description
-----|------------
`cartopy.feature.BORDERS` | Country boundaries
`cartopy.feature.COASTLINE` | Coastline, including major islands
`cartopy.feature.LAKES` | Natural and artificial lakes
`cartopy.feature.LAND` | Land polygons, including major islands
`cartopy.feature.OCEAN` | Ocean polygons
`cartopy.feature.RIVERS` | Single-line drainages, including lake centerlines
`cartopy.feature.STATES` | (limited to the United States at this scale)

Below we illustrate these features in a customized map of Africa. 

**Note**: the first time you run these commands will start the download of the Natural Earth Data shapefiles (at the best resolution). They will then be installed on your system (additional downloads may occur if you change the extent of the map, or set a given resolution).  

In [ ]:
import cartopy.feature as cfeature
import numpy as np

extent = [0, 40, 30, -40]
central_lon = np.mean(extent[:2])
central_lat = np.mean(extent[2:])

plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.AlbersEqualArea(central_lon, central_lat))
ax.set_extent(extent)

ax.add_feature(cfeature.OCEAN)
ax.add_feature(cartopy.feature.LAND, edgecolor='black')
ax.add_feature(cartopy.feature.LAKES, edgecolor='black')
ax.add_feature(cartopy.feature.RIVERS)
ax.gridlines(draw_labels=True)

If we want higher-resolution features, Cartopy can automatically download and create them from the [Natural Earth Data](http://www.naturalearthdata.com/) database 

In [ ]:
rivers_50m = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '50m')

plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.AlbersEqualArea(central_lon, central_lat))
ax.set_extent(extent)

ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.LAND, edgecolor='black')
ax.add_feature(rivers_50m, facecolor='None', edgecolor='b')
ax.gridlines()

Higher resolution coastlines are available with the [GSHHG dataset](https://www.ngdc.noaa.gov/mgg/shorelines/gshhs.html). To add GSHHG coastline data you need to use the specific feature `cartopy.feature.GSHHSFeature` and set the desired resolution *(yes, there is a typo... the database is called GSHHG and Cartopy calls it GSHHS...)*

The geography data come in five resolutions:
- **f**ull resolution: Original (full) data resolution (1:250,000). **Should only be used for small regions**
- **h**igh resolution: About 80 % reduction in size and quality.
- **i**ntermediate resolution: Another ~80 % reduction.
- **l**ow resolution: Another ~80 % reduction.
- **c**rude resolution: Another ~80 % reduction.

**Note**: Cartopy uses an older version of the dataset (v2.2), which is however sufficient for our purposes. If you want to use the [most recent version](http://www.soest.hawaii.edu/pwessel/gshhg/), you will need to download the shapefiles and use the dedicated python module. The download and setup of the GSHHG database _takes quite a long time_. Do not run the following cell during the online tutorial. Also note that if you are requesting the full resolution for a large region, the rendering will take some time and you may not see the difference.

In [ ]:
# plot a high resolution coastline of Tasmania
cl_high = cfeature.GSHHSFeature(scale='h')

extent = [144.5, 148.5, -40, -43.7]
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(extent)
ax.add_feature(cl_high)
ax.gridlines(draw_labels=True)

## Adding Data to the Map

Now that we know how to create a map, let's add our data to it! That's the whole point.

Because our map is a matplotlib axis, we can use all the familiar maptplotlib commands to make plots.
By default, the map extent will be adjusted to match the data. We can override this with the `.set_global` or `.set_extent` commands.

In [ ]:
# create some test data
new_york = dict(lon=-74.0060, lat=40.7128)
honolulu = dict(lon=-157.8583, lat=21.3069)
lons = [new_york['lon'], honolulu['lon']]
lats = [new_york['lat'], honolulu['lat']]

Key point: **the data also have to be transformed to the projection space**.
This is done via the `transform=` keyword in the plotting method. The argument is another `cartopy.crs` object.
If you don't specify a transform, Cartopy assume that the data is using the same projection as the underlying GeoAxis.

From the [Cartopy Documentation](https://scitools.org.uk/cartopy/docs/latest/tutorials/understanding_transform.html)

> The core concept is that the projection of your axes is independent of the coordinate system your data is defined in. The `projection` argument is used when creating plots and determines the projection of the resulting plot (i.e. what the plot looks like). The `transform` argument to plotting functions tells Cartopy what coordinate system your data are defined in.

In the following example, we plot the line between the two points on the Plate Carree projection, and compare it with the same line plotted on the `Geodetic` CRS, which is a spherical model of the Earth (geodesy is the science that accurately measure distances on the Earth). This is the so-called great-circle or orthodromic distance

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.plot(lons, lats, label='Equirectangular straight line')
ax.plot(lons, lats, label='Great Circle', transform=ccrs.Geodetic())
ax.coastlines()
ax.legend()
ax.set_global()

This method works if you know the coordinates of the locations of interest. You often need to display the principal cities, or to do computations of distances between known geolocations or other features. This can be done by using some web services called Geocoders, such as Google, Bing or the free (but limited) version of OpenStreetMap [Nominatim](https://nominatim.openstreetmap.org/ui/search.html).

[GeoPy](https://geopy.readthedocs.io/en/stable/) is a client that allows you to access different Geocoders. It can be used to compute the geodetic distance between two points (the real distance, which is the one you should always use for any computation. Note: geodesic is a noun, geodetic an adjective). The module `geopy` must be installed on your system. Open a terminal and type

`conda install -c conda-forge geopy`

before proceeding with the tutorial.
We will compute the distance between Cape Town and the South African Antarctic base SANAE IV

In [ ]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

In [ ]:
geolocator = Nominatim(user_agent='educational') # nominatim is a free service and requires an identificative agent
CT = geolocator.geocode('Cape Town') 
SANAE = geolocator.geocode('SANAE IV')
print(type(CT))
print(CT)
print(SANAE)
print('The distance is ',geodesic(CT.point,SANAE.point).km,' km')

The Location object has different attributes, and you can access the coordinates with `CT.latitude` and `CT.longitude`. 
The information can be used to plot the features on a map

In [ ]:
# a map of the Mediterranean, with a few places
extent = [-10, 30, 30, 40]
central_lon = np.mean(extent[:2])
central_lat = np.mean(extent[2:])

place = ['Athens','Algiers','Malta','Gibraltar']
address = []
for p in place:
    loc = geolocator.geocode(p,language="en")
    address.append(loc)
print(address)

plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.AlbersEqualArea(central_lon, central_lat))
ax.set_extent(extent)
ax.add_feature(cfeature.OCEAN)
ax.gridlines(draw_labels=True)
for p in range(len(place)):
    ax.text(address[p].longitude,address[p].latitude,place[p],transform=ccrs.Geodetic())

### Plotting 2D gridded and raster Data

The same principles apply to 2D data. Below we create some example data defined on a grid that uses regular lat / lon coordinates. The fake data are constructed using a 2D trigonometric function 
$$f(\phi,\lambda)=\cos 4\phi + \sin 4\lambda$$
which computes a value given a location pair $(\phi,\lambda)$ on the grid. 
In this way, we will generate *gridded data*. The data are contained in a 2D array (numpy `ndarray`), and they must be accompanied by two additional arrays of coordinates 

In [ ]:
import numpy as np
lon = np.linspace(-80, 80, 25)
lat = np.linspace(30, 70, 25)
lon2d, lat2d = np.meshgrid(lon, lat)
data = np.cos(np.deg2rad(lat2d) * 4) + np.sin(np.deg2rad(lon2d) * 4)
plt.contourf(lon2d, lat2d, data)
plt.colorbar()

Now we create a `PlateCarree` projection and plot the data on it without any `transform` keyword.
This happens to work because `PlateCarree` is the simplest projection of lat / lon data.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()
ax.contourf(lon, lat, data)

However, if we try the same thing with a different projection, we get the wrong result.

In [ ]:
projection = ccrs.RotatedPole(pole_longitude=-177.5, pole_latitude=37.5)
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.contourf(lon, lat, data)

To fix this, we need to pass the correct transform argument to `contourf`:

In [ ]:
projection = ccrs.RotatedPole(pole_longitude=-177.5, pole_latitude=37.5)
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.contourf(lon, lat, data, transform=ccrs.PlateCarree())

### Showing Images

Digital satellite images are also arrays, but they are usually called raster data

>In computer graphics and digital photography, a raster graphic is a dot matrix data structure that represents a generally rectangular grid of pixels, viewable via a computer display, paper, or other display medium. [Wikipedia](https://en.wikipedia.org/wiki/Raster_graphics)

This is still an array, but the data are stored as colour information (or multi-band values), instead of the real values. They can also be transformed to the real values (as for instance with ocean colour data that can be translated in chlorophyll concentration).

We can plot a satellite image on a map if *we know its extent*. Raster images are often available in the GeoTIFF format, which embeds the projection and coordinate information. They can be retrieved using the `gdal` tool available in the `osgeo` module.
We will first download the image of a tropical cyclone from space and then add it to a map

In [ ]:
# Remember that in python shell commands can be executed by putting the exclamation mark in front of the command
# You need to have wget installed on your system for this command to work
! wget https://lance-modis.eosdis.nasa.gov/imagery/gallery/2012270-0926/Miriam.A2012270.2050.2km.jpg

In [ ]:
fig = plt.figure(figsize=(8, 12))

# this example is taken from the cartopy docs. The image is on the Plate Carree projection
fname = 'Miriam.A2012270.2050.2km.jpg'
img_extent = (-120.67660000000001, -106.32104523100001, 13.2301484511245, 30.766899999999502)
img = plt.imread(fname)

ax = plt.axes(projection=ccrs.PlateCarree())

# set a margin around the data
ax.set_xmargin(0.05)
ax.set_ymargin(0.10)

# add the image. Because this image was a tif, the "origin" of the image is in the
# upper left corner
ax.imshow(img, origin='upper', extent=img_extent, transform=ccrs.PlateCarree())
ax.coastlines(resolution='50m', color='black', linewidth=1)

# mark a known place to help us geo-locate ourselves
ax.plot(-117.1625, 32.715, 'bo', markersize=7, transform=ccrs.Geodetic())
ax.text(-117, 33, 'San Diego', transform=ccrs.Geodetic())


## Doing More

Browse the [Cartopy Gallery](https://scitools.org.uk/cartopy/docs/latest/gallery/index.html) to learn about all the different types of data and plotting methods available!